In [9]:
cd 'C:\Users\barry\Documents\Academics\Quantecon-Notes'
addpath('C:\Users\barry\Documents\Academics\Quantecon-Notes\functions')

# Homework 2

## 1. Matching model

Consider the job matching model in Notes. Assume following quarterly parameterization: $\kappa = 0.19,\eta = 0.5, b = 0.4, \beta = 0.99, \delta = 0.02$. Assume aggregate matching function $m = Av^\alpha u^{1-\alpha}$ with $A=1.0, \alpha = 0.5$. Assume AR(1) process for log-productivity $\log{z}$ with $\rho =0.98$ and $\sigma =0.002$. 

**A. Approximate this AR(1) process using the Tauchen method with $N=10$ gird points.**

As in Tauchen (1986), for process $\log{z_t} = \rho \log{z_{t-1}} + \epsilon_t$, for 10 points the maximum grid $y_{10} = N(\frac{\sigma^2}{1-\rho^2})^{1/2} = 0.10$ and miminum grid $y_1 = -y_{10} = -0.10$. The other points are equally spaced. Let $d$ be the distance between each point $d = y_i - y_{i-1}$. The transition matrix is defined as 

$$\pi_{jk} = G(y_k + d/2 - \rho y_j) - G(y_k - d/2 - \rho y_j)$$
with boundaries 

$$\pi_{j1} = G(y_1 + d/2 - \rho y_j)$$
$$\pi_{jN} = 1-G(y_N - d/2 - \rho y_j)$$

where $G(x) = G(\epsilon_t < x)$ is the CDF for $\epsilon_t$

In [91]:
%%file functions/Tauchen.m

function [y,t_matrix] = Tauchen(N, sig, rho)
    m = 3;
    y_N = m*sqrt(sig^2/(1-rho^2));
    y = linspace(-y_N, y_N, N);
    
    d = y(2) - y(1);
    
    t_matrix = zeros(N,N);
    for j = 1:N
        for k = 1:N
            if k == 1
                t_matrix(j,k) = normcdf((y(1) + d/2-rho*y(j))/sig);
            elseif k == N
                t_matrix(j,k) = 1 - normcdf((y(N) - d/2 - rho*y(j))/sig);
            else
                t_matrix(j,k) = normcdf((y(k) + d/2 - rho*y(j))/sig) - normcdf((y(k) - d/2 - rho*y(j))/sig);
            end
        end
    end

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\Tauchen.m'.


In [94]:
[z, t_matrix] = Tauchen(10, 0.002, 0.98)


z =

   -0.0302   -0.0235   -0.0168   -0.0101   -0.0034    0.0034    0.0101    0.0168    0.0235    0.0302


t_matrix =

    0.9152    0.0848    0.0000    0.0000         0         0         0         0         0         0
    0.0281    0.8971    0.0749    0.0000    0.0000         0         0         0         0         0
    0.0000    0.0327    0.9015    0.0658    0.0000    0.0000         0         0         0         0
    0.0000    0.0000    0.0379    0.9044    0.0577    0.0000    0.0000         0         0         0
    0.0000    0.0000    0.0000    0.0438    0.9059    0.0503    0.0000         0         0         0
    0.0000    0.0000    0.0000    0.0000    0.0503    0.9059    0.0438    0.0000         0         0
    0.0000    0.0000    0.0000    0.0000    0.0000    0.0577    0.9044    0.0379    0.0000         0
    0.0000    0.0000    0.0000    0.0000    0.0000    0.0000    0.0658    0.9015    0.0327    0.0000
    0.0000    0.0000    0.0000    0.0000    0.0000    0.0000    0.0000 

**1. Derive analytically the key equation that determines $\theta(z)$ from the free entry condition**

$$\frac{\kappa}{\beta q(\theta(z_i))} = \sum_{j=1}^N [(1-\eta)(z_j - b) - \eta \kappa \theta(z_j) + \frac{(1-\eta)\kappa}{q(\theta(z_j))}]\Gamma(z_j, z_i)$$

where $q(\theta) = (Av^\alpha u^{1-\alpha})/v = A\theta^{-0.5} = \theta^{-0.5}$

**2. Solve for $\theta(z_j)$**



In [100]:
%%file functions/Theta_solve.m

function F = Theta_solve(theta)
    kappa = 0.19; eta = 0.5; b = 0.4; beta = 0.99; delta = 0.02; A = 1; alpha = 0.5; 
    rho = 0.98; sig = 0.002; N = 10;
    [z, t_matrix] = Tauchen(N, sig, rho);
    Z = exp(z);
    for i = 1:N
        F(i) = sum(((1-eta).*(Z- b)-eta*kappa*theta + ((1-eta)*kappa)./(A*theta.^(alpha-1)))*t_matrix(:,i)) - kappa/(beta*(A*theta(i)^(alpha - 1)));
    end

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\Theta_solve.m'.



Equation solved.

fsolve completed because the vector of function values is near zero
as measured by the value of the function tolerance, and
the problem appears regular as measured by the gradient.


theta =

    1.5694    1.7362    1.7554    1.7807    1.8063    1.8323    1.8587    1.8853    1.9190    1.7887




**3. Use data from FRED to calculate the HP-filtered ($\lambda = 1600$) standard deviations of the quarterly time series of the log of $\{v,u,\theta, w\}$**

**4. Simulate the economy for 10,500 periods. Drop the first 500. Compute the standard deviation of vacancies, unemployment, market tightness.**

Recall the law of motion $$u_{t+1} = u_t - p(\theta_t) + \delta (1-u_t)$$
and the equation for wage from Nash bargaining: $$w_t = \eta z + (1-\eta)b + \eta \kappa \theta(z)$$

In [186]:
%%file functions\z_recursive_sample.m

function z_index = z_recursive_sample(z_index_prev, t_matrix)
    
    dist = t_matrix(z_index_prev,:);
    cdf = cumsum(dist);
    seed = rand;
    z_index = 1;
    for i = 2:length(t_matrix)
        if seed < cdf(i)
            z_index = i;
            break
        end
    end

Created file 'C:\Users\barry\Documents\Academics\Quantecon-Notes\functions\z_recursive_sample.m'.


In [358]:
options = optimset('Display','off');


N = 10;
sig = 0.002;
rho = 0.98;
A = 1;
alpha = 0.5;
delta = 0.02;
eta = 0.5;
b = 0.4;
kappa = 0.19;

T = 100;
[z, t_matrix] = Tauchen(N, sig, rho);
Z = exp(z);
z_index_0 = randi(10);

z_index = zeros(T,1);
u = zeros(T,1);
v = zeros(T,1);
w = zeros(T,1);
theta = zeros(T,1);

theta_0 = linspace(1,4,10);

fun = @Theta_solve;

z_index(1) = z_index_0;
u(1) = 0.036; %unemployment rate 3.6%

for i = 1:T-1
    
    theta_val = fsolve(fun, theta_0, options);
    theta(i) = theta_val(z_index(i));
    v(i) = theta(i)*u(i);
    w(i) = eta*Z(z_index(i)) + (1-eta)*b + eta*kappa*theta(i);
    
    u(i+1) = u(i) - A*theta(i)^(1-alpha)*u(i)+ delta*(1-u(i));
    z_index(i+1) = z_recursive_sample(z_index(i), t_matrix);
    end

In [361]:
std(u)
std(v)
std(theta)
std(w)


ans =

    0.0023


ans =

    0.0050


ans =

    0.1820


ans =

    0.0872




Shimer Puzzle: given a reasonable productivity shock, the variation in unemployment predicted from DMP model is too little comparing with the real data. 

## 2. Deterministic OLG model

Model setup: individual lives for $I$ periods: $i=1,2,\cdots I$. The first $I^w$ periods he works then he retires. Preference over comsumption and hours worked are 

$$
\sum_{i=1}^I \beta^{i-1}[\frac{c_i^{1-\gamma}}{1-\gamma} - \phi \frac{h_i^{1+\eta}}{1+\eta}]
$$

His productivity when working is $\epsilon_i = (1+g)^{i-1}$.

A retiree earns a fraction $\rho$ of her average earnings (wage times hours $h$ times productivity $\epsilon$) during her working life. The individual is born with zero asset and can save and borrow in a risk-free bond $a$ at the interest rate $r$. A no-Ponzi condition ensures that $a_{I+T} \geq 0$. A tax $\tau$ is levied on labor income by the government. 

**A. Model solution**

The optimization program of the individual is 

$$
\begin{aligned}
\max_{c_i, h_i} &\sum_{i=1}^I \beta^{i-1}[\frac{c_i^{1-\gamma}}{1-\gamma} - \phi \frac{h_i^{1+\eta}}{1+\eta}] \\
s.t &a_{i+1} = (1+r)a_i + w(1+g)^{i-1}h_i - c_i \text{if  } i\leq I^w \\
&a_{i+1} = (1+r)a_i + \frac{\rho}{I^w} w\sum_{i=1}^{I^w}(1+g)^{i-1}h_i - c_i \text{if  } i> I^w 
\end{aligned}$$

For $i \leq I^w$, the Euler equation requires 

$$
u'(c_i) = c_i^{-\gamma} = \beta(1+r)u'(c_{i+1}) = \beta (1+r) c_{i+1}^{-\gamma}
$$
$$
u'(h_i) = -\phi h_i^{\eta} =  \frac{\beta (1+r)}{1+g} u'(h_{i+1}) = -\phi \frac{\beta (1+r)}{1+g} h_{i+1}^{\eta}
$$

So 
$(\frac{c_{i+1}}{c_i})^\gamma = \beta (1+r)$, $(\frac{h_{i+1}}{h_i})^{-\eta} = \frac{\beta (1+r)}{1+g}$

From intertemporal tradeoff, we have $h_i^\eta c_i^\gamma = \frac{1}{\phi} (1+g)^{i-1}w$

So the consumption and working hour path are given by 

$$\begin{aligned}
c_i &= (\beta(1+r))^{i/\gamma}c_0 \\
h_i &= (\frac{1+g}{\beta(1+r)})^{i/\eta}h_0 \\ 
c_0^\gamma h_0^{\eta} &= w/\phi
\end{aligned}$$

Now therefore the intertemporal transfer of asset after $I_w$ is 

$$
\begin{aligned}
a_{i+1} &= (1+r)a_i + \frac{\rho}{I^w} w\sum_{i=1}^{I^w}(1+g)^{i-1}h_i - c_i \\
 &= (1+r)a_i + \frac{\rho}{(1+g)I_w} \frac{1-(1+g)^{(1+1/\eta)I_w}(\frac{1}{\beta (1+r)})^\frac{I_w}{\eta}}{(1+g)^{(1+1/\eta)} (\frac{1}{\beta (1+r)})^\frac{1}{\eta}}h_0 - c_i
 \end{aligned}
$$

And the consumption path is still governed by $c_i = (\beta(1+r))^{i/\gamma}c_0 $

So the lifetime utility is 

$$
\begin{aligned}
&\sum_{i=1}^I \beta^{i-1} [\frac{(\beta (1+r))^{\frac{1-\gamma}{\gamma}i}c_0^{1-\gamma}}{1-\gamma} - \phi \frac{(\frac{1+g}{\beta(1+r)})^{\frac{1+\eta}{\eta}i}h_0^{1+\eta}}{1+\eta}] \\
=& \sum_{i=1}^I \beta^{i-1} \frac{(\beta (1+r))^{\frac{1-\gamma}{\gamma}i}c_0^{1-\gamma}}{1-\gamma} - \sum_{i=1}^{I_w} \beta^{i-1}\phi \frac{(\frac{1+g}{\beta(1+r)})^{\frac{1+\eta}{\eta}i}(\frac{w}{\phi})^{\frac{1+\eta}{\eta}}c_0^{-\frac{1+\eta}{\eta}\gamma}}{1+\eta} \\ 
=&\frac{c_0^{1-\gamma}}{(1-\gamma)\beta}\sum_{i=1}^I (\beta^{1/\gamma} (1+r)^{\frac{1-\gamma}{\gamma}})^i - \frac{w^{\frac{1+\eta}{\eta}}\phi^{-1/\eta}c_0^{-\frac{1+\eta}{\eta}\gamma}}{\beta(1+\eta)} \sum_{i=1}^{I_w} [(\frac{1+g}{1+r})^{\frac{1+\eta}{\eta}}\beta^{-\frac{1}{\eta}}]^i
\end{aligned}
$$

which is increasing in $c_0$ and therefore optimize to the boundary. The constraint is given by no-ponzi condition, 